# Object Detection and OCR Evaluation Pipeline
This notebook demonstrates a complete pipeline for object detection using YOLOv5 and Optical Character Recognition (OCR) using EasyOCR. The steps include performing object detection, cropping detected objects, applying OCR, and evaluating the OCR results.

## 1. Setting Up the Environment
We start by importing the necessary libraries, including YOLOv5 for object detection, EasyOCR for OCR, OpenCV for image processing, and Pandas for data manipulation.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import torch
import os
import yaml
import cv2
import json
import pandas as pd
import random
import easyocr
import cv2
import matplotlib.pyplot as plt

We define the directories for the YOLOv5 model weights, dataset, and output locations.

In [2]:
project_dir = os.path.dirname(os.getcwd())
yolo_location = os.path.join(project_dir, 'yolov5')
dataset_location = os.path.join(project_dir, 'data', 'ocr_evaluation')
weights_location = os.path.join(project_dir, 'yoloV5_model_weights')

output_location = os.path.join(project_dir, 'data', 'yolo_v5')
output_name = 'detection_results'

## 2. Performing Object Detection


We run the YOLOv5 detection script to detect objects in the dataset images. The results are saved in the specified output directory.

In [3]:
%cd {yolo_location}
!python detect.py --weights {weights_location}/best.pt --img 416 --conf 0.4 --source {dataset_location} --save-txt --save-conf --project {output_location} --name {output_name}

/home/roma/5sem/Real-time_Number_Plate_Recognition/yolov5


/home/roma/5sem/Real-time_Number_Plate_Recognition/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


detect: weights=['/home/roma/5sem/Real-time_Number_Plate_Recognition/yoloV5_model_weights/best.pt'], source=/home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5, name=detection_results, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2024-10-21 Python-3.12.7 torch-2.5.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5834MiB)

Fusing layers... 
YOLOv5s_custom summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/395 /home/roma/5sem/Real-time_Number_Plate_Recognition/data/ocr_evaluation/00000.jpg: 320x416 1 License_Plate, 35.9ms


We iterate over the detected objects, crop them from the source images, and save the cropped images.

In [4]:
input_img_dir = dataset_location
input_txt_dir = f"{output_location}/{output_name}/labels"
output_dir = f'{output_location}/cropped_plates'
os.makedirs(output_dir, exist_ok=True)

for img_file in os.listdir(input_img_dir):
    if img_file.endswith('jpg'):
        img_path = os.path.join(input_img_dir, img_file)
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]

        txt_path = os.path.join(input_txt_dir, img_file[:-3]+'txt')
        try:
            with open(txt_path, 'r') as f:
                max_conf = -1.0
                for line in f.readlines():
                    class_id, x_center, y_center, width, height, conf = map(float, line.split())
                    x_center *= img_width
                    y_center *= img_height
                    width *= img_width
                    height *= img_height

                    x1 = int(x_center - width / 2)
                    y1 = int(y_center - height / 2)
                    x2 = int(x_center + width / 2)
                    y2 = int(y_center + height / 2)
                    cropped_plate = img[y1:y2, x1:x2]

                    # save plate with the highest confidence
                    if conf > max_conf:
                        plate_save_path = os.path.join(output_dir, f'plate_{img_file}')
                        cv2.imwrite(plate_save_path, cropped_plate)
                        max_conf = conf
        except FileNotFoundError:
            print(f"File not found: {txt_path}")

File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/00591.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/01026.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/01803.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/01003.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/00440.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/01595.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/00573.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/detection_results/labels/00741.txt
File not found: /home/roma/5sem/Real-time_Number_Plate_Recognition/data/yolo_v5/